In [2]:
# Importar las librerías necesarias
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Parámetros de la red
EPOCHS = 50
IMAGE_SIZE = (224, 224)
INPUT_SHAPE = (224, 224, 3)
SEED = 123
BATCH_SIZE = 32
BUFFER_SIZE = 250
LEARNING_RATE = 0.001

# Directorio de imágenes
images_dir = '../arcgis-survey-images'

# Cargar dataset de imágenes
train_ds = image_dataset_from_directory(
    images_dir,
    labels="inferred",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    shuffle=True
)

validation_ds = image_dataset_from_directory(
    images_dir,
    labels="inferred",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=SEED
)

# Obtener los nombres de las clases
class_names = train_ds.class_names

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

# Aplicar data augmentation al conjunto de entrenamiento
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

# Cache y prefetch para mejorar el rendimiento
train_ds = train_ds.cache().shuffle(BUFFER_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Importar EfficientNetB5
from tensorflow.keras.applications import EfficientNetB5

# Cargar el modelo base (EfficientNetB5)
base_model = EfficientNetB5(input_shape=INPUT_SHAPE,
                            include_top=False,
                            weights='imagenet')

# Congelar las capas del modelo base
base_model.trainable = False

# Construir el modelo
inputs = tf.keras.Input(shape=INPUT_SHAPE)
x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)  # Añadir Dropout para evitar overfitting
outputs = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=EPOCHS,
)

# Descongelar algunas capas para fine-tuning
base_model.trainable = True
fine_tune_at = 300  # Descongelar desde la capa 300

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompilar el modelo con un learning rate más pequeño
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continuar el entrenamiento (fine-tuning)
fine_tune_epochs = 20
total_epochs = EPOCHS + fine_tune_epochs

history_fine = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1]
)

# Gráfica de la pérdida y precisión
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']

loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Precisión de entrenamiento')
plt.plot(val_acc, label='Precisión de validación')
plt.legend(loc='lower right')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.title('Precisión de entrenamiento y validación')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Pérdida de entrenamiento')
plt.plot(val_loss, label='Pérdida de validación')
plt.legend(loc='upper right')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.title('Pérdida de entrenamiento y validación')
plt.show()

# Evaluación en el conjunto de validación
loss, accuracy = model.evaluate(validation_ds)
print(f"Pérdida en validación: {loss:.4f}")
print(f"Precisión en validación: {accuracy:.4f}")


Found 3289 files belonging to 5 classes.
Using 2632 files for training.
Found 3289 files belonging to 5 classes.
Using 657 files for validation.
115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 237s 2s/step - accuracy: 0.2831 - loss: 1.5389 - val_accuracy: 0.3349 - val_loss: 1.4648
Epoch 2/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step - accuracy: 0.3123 - loss: 1.4997 - val_accuracy: 0.3090 - val_loss: 1.4788
Epoch 3/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.3142 - loss: 1.5025 - val_accuracy: 0.3531 - val_loss: 1.4700
Epoch 4/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.3191 - loss: 1.5150 - val_accuracy: 0.3531 - val_loss: 1.4577
Epoch 5/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 206s 2s/step - accuracy: 0.3071 - loss: 1.4989 - val_accuracy: 0.3425 - val_loss: 1.4666
Epoch 6/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.3167 - loss: 1.5023 - val_accuracy: 0.3242 - val_loss: 1.4677
Epoch 7/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 19

KeyboardInterrupt: 